In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
import transformers
transformers.__version__

'4.34.1'

In [3]:
#!sudo pip install -q accelerate peft==0.4.0 bitsandbytes trl==0.4.7

In [4]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch

In [5]:
df = pd.read_csv('/data/mistral/query-to-mql/training-data.csv')

In [6]:
df.columns

Index(['Query', 'MQL', 'Rationale'], dtype='object')

In [7]:
df.shape

(37, 3)

In [8]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# The instruction dataset to use
#dataset_name = ""

# Fine-tuned model name
#new_model = "mistral-ft-peft-on-template_and_user_query-data"

In [9]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [10]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

In [62]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/data/mistral/query-to-mql/oct-27"

# Number of training epochs
num_train_epochs = 15

# Enable fp16/bf16 training (set bf16 to True with an A100)
# fp16 = False
fp16 = True # not using quantisation
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True


# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 200

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 20

# Log every X updates steps
logging_steps = 20

In [54]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [13]:
promt_template_measure = """<s>[INST]<<SYS>>
Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.

<</SYS>>
[/INST]
[MQL]
{mql}
[/MQL]

the steps and rationale used to achieve above structured output is as below.
{rationale}
</s>
"""

In [14]:
df.columns

Index(['Query', 'MQL', 'Rationale'], dtype='object')

In [15]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

In [16]:
date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [17]:
def create_fine_tuning_dataset(row):
    mql = row['MQL']
    user_query = row['Query']
    rationale = row['Rationale']
    formated = promt_template_measure.format(context=context,
                                             date_input=date_input,
                                             user_query=user_query,
                                             mql=mql,
                                             rationale=rationale)
    return formated

In [18]:
df['fine_tuning_dataset']=df.apply(create_fine_tuning_dataset, axis=1)

In [21]:
df.drop(columns=['Query', 'MQL', 'Rationale'], inplace=True)
df.shape

(37, 1)

In [22]:
train_dataset = Dataset.from_pandas(df)

In [23]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 37
})

In [25]:
train_dataset['fine_tuning_dataset'][0]

'<s>[INST]<<SYS>>\nGiven the context : {\n    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},\n                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},\n                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},\n                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},\n                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},\n                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},\n                  {"ENTITY": "Country", "other names": ["country", "countries"]}],\n    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},\n               {"ENTITY": "Phone", "other names": ["phone", "phones

In [26]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [27]:
bnb_4bit_quant_type

'nf4'

In [28]:
compute_dtype

torch.float16

In [29]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [30]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [31]:
!sudo pip install -q pynvml

In [32]:
from pynvml.smi import nvidia_smi
nvsmi = nvidia_smi.getInstance()
nvsmi.DeviceQuery('memory.free, memory.total')

{'gpu': [{'fb_memory_usage': {'total': 16384.0,
    'free': 15972.9375,
    'unit': 'MiB'}}]}

In [33]:
#!df -H

In [34]:
torch.cuda.is_available()

True

In [35]:
# del model
# torch.cuda.empty_cache()

In [36]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
#     torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [39]:
max([len(tokenizer.encode(df['fine_tuning_dataset'][i])) for i in range(df.shape[0])])

1600

In [55]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules = ["q_proj", "v_proj"],
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [63]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
#     eval_steps=50, # requires when eval_dataset is defined
#     per_device_eval_batch_size=1, # Batch size for evaluation
#     evaluation_strategy="steps", # requires when eval_dataset is defined
    logging_strategy="steps",
    logging_steps=5,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=200,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
#     load_best_model_at_end=True,
#     save_total_limit=1,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False
)

In [64]:
## Getting FLOPs of model

model_flops = (
  model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, 2048)
      )
    }
  )
  * training_arguments.gradient_accumulation_steps
)

#print(model)
print("Memory footprint", model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

Memory footprint 5.185232896 GB
Flops 87710.798708736 GFLOPs


In [58]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 37
})

In [57]:
#print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [65]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
#     eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="fine_tuning_dataset",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/opt/conda/lib/python3.8/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/37 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

Step,Training Loss
1,1.035000
2,0.978700
3,0.975200
4,0.995600
5,0.893700
6,0.927600
7,0.885500
8,0.790100
9,0.799600
10,0.758100


In [ ]:
# Fine-tuned model name
new_model_name = "mistral-ft-peft-v1-lr-64-with-more-data"

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [59]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj)

In [12]:
from peft import PeftModel, PeftConfig

In [5]:
new_model_name = "/data/mistral/query-to-mql/0ct-25/checkpoint-800"

In [6]:
#del model
torch.cuda.empty_cache()

In [7]:
#del model
torch.cuda.empty_cache()

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(new_model_name, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

In [83]:
# output_merged_dir = os.path.join(new_model_name, "final_merged_checkpoint")
# model.save_pretrained(output_merged_dir, safe_serialization=True)

In [8]:
query_template_v1 = """<s>[INST]<<SYS>>
Given the CONTEXT:{context}, convert the 'user query' into JSON format which captures basic measures asked by user and maps it to CONTEXT.

<</SYS>>
User query : {user_query}

Converted JSON is as shown below: 
[/INST]
[MEASUREMQL]
"""

In [9]:
#model_to_merge.to('cuda')

In [22]:
df['query'][0],df['measure'][0],df['measure_mql'][0]

('trend of profit',
 "{'measure': 'PROFIT', 'measure_label': 'Profit in Dollars'}",
 "{'measure': {'profit': {'PROFIT': {'label': 'Profit in Dollars', 'order': 'desc', 'operator': 'sum'}}}}")

In [10]:
def predict_template_query_v1(user_query, context):
    inp = query_template_v1.format(context=context,
                                   user_query=user_query)
    _inputs = tokenizer.encode(inp, return_tensors="pt")
    outputs = model.generate(input_ids=_inputs.to('cuda'), max_length= 256, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[MEASUREMQL]\n')[1]
    return output_new.split('\n[/MEASUREMQL]')[0]

### Testing on data used for query to intermediate using openai prompt base approach

In [15]:
%%time
i=0
user_query = 'show me the 2 top segments basis sales'
print('user query: ', user_query)
context = "{'measure': ['SCHEDULEDHOURS', 'Sales', 'PROFIT'], 'measure_label': ['Number Requests For Shift','PROFIT']}"
output = predict_template_query_v1(user_query=user_query,context=context)
output

user query:  show me the 2 top segments basis sales
CPU times: user 6.41 s, sys: 218 ms, total: 6.63 s
Wall time: 6.63 s


"{'measure': None}"

In [17]:
%%time
i=0
user_query = 'top 2 and bottom 3 segments by sales'
print('user query: ', user_query)
context = "{'measure': ['SCHEDULEDHOURS', 'Sales', 'PROFIT'], 'measure_label': ['Number Requests For Shift','Sales']}"
output = predict_template_query_v1(user_query=user_query,context=context)
output

user query:  top 2 and bottom 3 segments by sales
CPU times: user 6.46 s, sys: 174 ms, total: 6.64 s
Wall time: 6.64 s


"{'measure': None}"

In [18]:
%%time
i=0
user_query = 'top 2 segments and bottom 3 sub-category basis quantity'
print('user query: ', user_query)
context = "{'measure': ['SCHEDULEDHOURS', 'Quantity',], 'measure_label': ['Quantity in millions']}"
output = predict_template_query_v1(user_query=user_query,context=context)
output

user query:  top 2 segments and bottom 3 sub-category basis quantity
CPU times: user 8.19 s, sys: 191 ms, total: 8.38 s
Wall time: 8.38 s


"{'measure': None}"

In [ ]:
%%time
i=0
user_query = 'top 2 segments and bottom 3 sub-category basis quantity'
print('user query: ', user_query)
context = "{'measure': ['SCHEDULEDHOURS', 'Quantity',], 'measure_label': ['Quantity in millions']}"
output = predict_template_query_v1(user_query=user_query,context=context)
output

In [25]:
%%time
user_query = 'why did profit changed in may 2023'
print('user query: ', user_query)
context = "{'measure': ['PROFIT'], 'measure_label': ['Number Requests For Shift','PROFIT']}"
output = predict_template_query_v1(user_query=user_query,context=context)
output

user query:  why did profit changed in may 2023
CPU times: user 8.5 s, sys: 184 ms, total: 8.68 s
Wall time: 8.69 s


"{'measure': None}"

In [24]:
%%time
i=0
user_query = df['query'][i]
print('user query: ', user_query)
context = df['measure_new'][i]
output = predict_template_query_v1(user_query=user_query,context=context)
output, df['measure_mql'][i]

user query:  trend of profit


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


CPU times: user 7.88 s, sys: 220 ms, total: 8.1 s
Wall time: 8.11 s


("{'measure': {'profit': {'PROFIT': {'label': 'Profit in Dollars', 'order': 'desc', 'operator': 'sum'}}}}",
 "{'measure': {'profit': {'PROFIT': {'label': 'Profit in Dollars', 'order': 'desc', 'operator': 'sum'}}}}")

In [25]:
%%time
i=2
user_query = df['query'][i]
print('user query: ', user_query)
context = df['measure_new'][i]
output = predict_template_query_v1(user_query=user_query,context=context)
output, df['measure_mql'][i]

user query:  why profit change in july 2022
CPU times: user 7.12 s, sys: 261 ms, total: 7.38 s
Wall time: 7.38 s


("{'measure': {'profit': {'PROFIT': {'label': 'Profit in Dollars', 'order': 'desc', 'operator': 'sum'}}}}",
 "{'measure': {'profit': {'PROFIT': {'label': 'Profit in Dollars', 'order': 'desc', 'operator': 'sum'}}}}")

In [26]:
train_df.columns

Index(['Unnamed: 0', 'query', 'measure_new', 'measure_mql'], dtype='object')

In [27]:
train_df.rename(columns={'Unnamed: 0':'index'}, inplace=True)

In [28]:
train_df.columns

Index(['index', 'query', 'measure_new', 'measure_mql'], dtype='object')

In [29]:
df.shape[0]

17334

In [30]:
untrained_index = set(range(df.shape[0]))-set(train_df['index'].tolist())

In [31]:
%%time
i=2000
user_query = df['query'][i]
print('user query: ', user_query)
context = df['measure_new'][i]
output = predict_template_query_v1(user_query=user_query,context=context)
output, df['measure_mql'][i]

user query:  what is growth of number of providers across arizona
CPU times: user 5.44 s, sys: 204 ms, total: 5.64 s
Wall time: 5.64 s


("{'measure': None}", "{'measure': None}")

In [32]:
import random
from tqdm import tqdm

In [33]:
untrained_500_random = random.choices(list(untrained_index), k=500)

In [44]:
prediction = []
for i in tqdm(untrained_500_random):
    user_query = df['query'][i]
    context = df['measure_new'][i]
    output = predict_template_query_v1(user_query=user_query,context=context)
    prediction.append([df['measure_mql'][i],output])

100%|██████████| 500/500 [58:00<00:00,  6.96s/it] 


In [47]:
prediction[0][1]

"{'measure': None}"

In [48]:
pred_actual = []
pred_pred = []

for p in prediction:
    pred_actual.append(p[0])
    pred_pred.append(p[1])

In [55]:
df.columns

Index(['query', 'mql', 'account_id', 'metadata', 'measure', 'dimension',
       'derived_measure', 'date', 'measure_mql', 'dimension_mql', 'action_mql',
       'date_mql', 'measure_new'],
      dtype='object')

In [59]:
type(untrained_500_random)

list

In [62]:
pred_query = list(df['query'][untrained_500_random])
pred_measure = list(df['measure_new'][untrained_500_random])

In [63]:
pred_df = pd.DataFrame()

In [65]:
pred_df['query'] = pred_query
pred_df['measure_metadata'] = pred_measure
pred_df['actual_mql'] = pred_actual
pred_df['pred_mql'] = pred_pred

In [82]:
pred_df.to_csv('/data/mistral/query-to-mql/0ct-25/pred_df.csv', index=False)

In [74]:
correct = []
for i,row in pred_df.iterrows():
    if row['actual_mql']==row['pred_mql']:
        correct.append(1)
    else:
        correct.append(0)

In [75]:
pred_df['correct']=correct

In [84]:
pred_df

,query,measure_metadata,actual_mql,pred_mql,correct
0,what are fill rate in aug 2022 for cna,"{'measure': ['FSP_LONGITUDE', 'FILL_RATE_ASSIG...",{'measure': None},{'measure': None},1
1,what is trend of number of requested shifts,"{'measure': ['AFS_NUMBER_REQUESTS_FOR_SHIFT', ...",{'measure': None},{'measure': None},1
2,what is worked hours in quarter2 2022 vs quart...,"{'measure': ['SRD_RATE', 'FPP_LATITUDE', 'AFS_...",{'measure': {'worked hours': {'AFS_WORKED_HOUR...,{'measure': {'worked hours': {'AFS_WORKED_HOUR...,1
3,what is monthly growth rate of number of facil...,"{'measure': ['FILL_RATE_OPENED', 'Active Facil...",{'measure': None},{'measure': None},1
4,what is daily trend of number of facility in 2023,"{'measure': ['FFP_LONGITUDE', 'FOP_LATITUDE'],...",{'measure': None},{'measure': None},1
...,...,...,...,...,...
495,what will be worked hours in next 6 months,"{'measure': ['FFP_NUM_TEAM_MEMBERS', 'AFS_DEFA...",{'measure': {'worked hours': {'AFS_WORKED_HOUR...,{'measure': {'worked hours': {'AFS_WORKED_HOUR...,0
496,what is trend of profit,"{'measure': ['OPENED_HOURS', 'AFS_CONTRACT_RAT...",{'measure': {'profit': {'PROFIT': {'label': 'P...,{'measure': {'profit': {'PROFIT': {'label': 'P...,1
497,top 5 state contributing to number of provider...,"{'measure': ['FFP_LATITUDE', 'PROFIT'], 'measu...",{'measure': None},{'measure': None},1
498,what is monthly trend of filled hours in texas...,"{'measure': ['FPP_LATITUDE', 'COUNT(DISTINCT F...",{'measure': None},{'measure': None},1


In [81]:
for i, k in pred_df[pred_df['correct']==0]

,actual_mql,actual_mql
20,{'measure': {'profit': {'PROFIT': {'label': 'P...,{'measure': {'profit': {'PROFIT': {'label': 'P...
28,{'measure': {'worked hours': {'AFS_WORKED_HOUR...,{'measure': {'worked hours': {'AFS_WORKED_HOUR...
32,{'measure': {'worked hours': {'AFS_WORKED_HOUR...,{'measure': {'worked hours': {'AFS_WORKED_HOUR...
35,{'measure': {'worked hours': {'AFS_WORKED_HOUR...,{'measure': {'worked hours': {'AFS_WORKED_HOUR...
36,{'measure': {'worked hours': {'AFS_WORKED_HOUR...,{'measure': {'worked hours': {'AFS_WORKED_HOUR...
...,...,...
459,{'measure': {'profit': {'PROFIT': {'label': 'P...,{'measure': {'profit': {'PROFIT': {'label': 'P...
462,{'measure': {'profit': {'PROFIT': {'label': 'P...,{'measure': {'profit': {'PROFIT': {'label': 'P...
478,{'measure': {'worked hours': {'AFS_WORKED_HOUR...,{'measure': {'worked hours': {'AFS_WORKED_HOUR...
480,{'measure': {'worked hours': {'AFS_WORKED_HOUR...,{'measure': {'worked hours': {'AFS_WORKED_HOUR...


In [17]:
prompt = query_template_v2.format(user_query='brands least profitable in 2021')

In [18]:
tokens = tokenizer.encode(prompt, return_tensors="pt")

In [22]:
%%time
outputs = model.generate(input_ids=tokens.to('cuda'), max_length= 180, )

CPU times: user 6.61 s, sys: 135 ms, total: 6.75 s
Wall time: 6.75 s


In [23]:
tokenizer.decode(outputs[0])

'<s>[INST]<<SYS>>\nYou are an advanced template converter that converts user question to a specific template which answers the user question.\n\n<</SYS>>\n\nbrands least profitable in 2021\n[/INST]\n[LUMINTEMPLATE]\nList of brands with lowest profit in 2021\n[/LUMINTEMPLATE]\n\nWhich are the top 5 brands based on profit share in 2021\n[/LUMINTEMPLATE]\n\nWhich are the top 5 brands based on market share in 2021\n[/LUMINTEMPLATE]\n\nWhich are the top 5 brands based on sales share in 2021\n[/LUMINTEMPLATE]\n\nWhich are the top 5'